# Mission to Mars
---
Webscrapper for NASA's Mars Exploration Program website.
---
### Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from os.path import basename
import time
import pandas as pd

In [2]:
# Splinter set up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

---
### Latest News
- Grab titles and teaser text of all the latest news.

In [3]:
# Open browser with splinter for scraping
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
mars_soup = BeautifulSoup(html, 'html.parser')


In [4]:
# Select the container for main contents > article containers
news_hits = mars_soup.find('div', class_='grid_list_page module content_page')\
    .find_all('div', class_='image_and_description_container')


In [5]:
# Create lists for findings
news_titles = []
news_teaser = []

# Go through each article and strip the title and preview summary
for news in news_hits:
    title = news.find('div', class_='content_title').get_text()
    news_titles.append(title)
    tease = news.find('div', class_='article_teaser_body').get_text()
    news_teaser.append(tease)

# Check the count of items scraped and preview the last find
print(f'{len(news_titles)} titles found.\n{len(news_teaser)} teasers found.')
print(f'Last Scraped Result: "{title}" \n    {tease}')

40 titles found.
40 teasers found.
Last Scraped Result: "Drilling Success: Curiosity is Collecting Mars Rocks" 
    Engineers will now test delivering samples to instruments inside NASA's Curiosity Mars rover.


---
### JPL Mars Space Images
- Grab the full size jpeg of the featured image
- Retain a copy of the URL to the featured image

In [6]:
# Redirect browser to the JPL website and prepare for scraping
base_url = 'https://www.jpl.nasa.gov' 
url = base_url + '/spaceimages/?search=&category=Mars'
browser.visit(url)

In [7]:
# Have splinter focus on the full size of the featured image
browser.click_link_by_partial_text('FULL IMAGE')

# Pause to give browser time to load before moving on
time.sleep(5)

In [8]:
# Scrape the URL of the image
html = browser.html
jpl_soup = BeautifulSoup(html, 'html.parser')
feature_img_tag = jpl_soup.find('img', class_='fancybox-image')
feature_img_url = feature_img_tag.get('src')
print(base_url + feature_img_url)

# Grab the immage itself
save_dir = basename(feature_img_url)
featured_img = requests.get(base_url+feature_img_url).content
with open(save_dir, 'wb') as dl:
    dl.write(featured_img)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18907_ip.jpg


---
### Mars Weather
- Grab the latest Mars weather status tweet

In [9]:
# Navigate to mars weather report twitter page
twit_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twit_url)

In [10]:
# Grab html
html = browser.html
twit_soup = BeautifulSoup(html, 'html.parser')

In [11]:
# Grab first tweet for weather status
tweet = twit_soup.find('div', class_='js-tweet-text-container').get_text()
print(tweet)


Sol 2223 (2018-11-07), high 2C/35F, low -64C/-83F, pressure at 8.50 hPa, daylight 06:18-18:35



---
### Mars Facts
- Scrape the Mars facts into a dataframe

In [12]:
fact_url = 'http://space-facts.com/mars/'
browser.visit(fact_url)
html = browser.html
fact_soup = BeautifulSoup(html, 'html.parser')

In [13]:
facts_html = pd.read_html(fact_url)
facts_pd = facts_html[0]
facts_pd.columns

Int64Index([0, 1], dtype='int64')

In [14]:
facts_pd.columns = ['Planet Profile', 'Mars']
facts_pd.set_index('Planet Profile', inplace=True)
facts_pd

,Mars
Planet Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
facts = fact_soup.find('div', id='facts').find('ul').find_all('li')


In [16]:
# Containers for fact portions
fact_head = []
fact_body = []

# Scrape for facts
for fact in facts:
    head_sp = fact.find('strong')
    fact_head.append(head_sp.get_text())
    body = fact.get_text().split('\n')[1]
    fact_body.append(body)

# Check scrape results
print(f'{len(fact_head)} / {len(fact_body)}')
    
about_mars_pd = pd.DataFrame({'Fact':fact_head, 'Explanation':fact_body})
about_mars_pd

9 / 9


,Fact,Explanation
0,Mars and Earth have approximately the same lan...,Even though Mars has only 15% of the Earth’s v...
1,Mars is home to the tallest mountain in the so...,"Olympus Mons, a shield volcano, is 21km high a..."
2,Only 18 missions to Mars have been successful.,As of September 2014 there have been 40 missio...
3,Mars has the largest dust storms in the solar ...,They can last for months and cover the entire ...
4,On Mars the Sun appears about half the size as...,"At the closest point to the Sun, the Martian s..."
5,Pieces of Mars have fallen to Earth.,Scientists have found tiny traces of Martian a...
6,Mars takes its name from the Roman god of war.,"The ancient Greeks called the planet Ares, aft..."
7,There are signs of liquid water on Mars.,For years Mars has been known to have water in...
8,One day Mars will have a ring.,In the next 20-40 million years Mars’ largest ...


In [17]:
rel_sp = fact_soup.find('div', class_='yarpp-related').find_all('a')

In [18]:
# Containers for related fact portions
rel_head = []
rel_body = []
rel_link = []

# Scrape for facts
for fact in rel_sp:
    rel_head.append(fact.get_text())
    rel_link.append(fact['href'])
    rel_body.append(fact.next_sibling)

# Check scrape results
print(f'{len(rel_head)} / {len(rel_body)} / {len(rel_link)}')
    
related_pd = pd.DataFrame({'Title':rel_head, 'Teaser':rel_body, 'Link':rel_link})
related_pd

3 / 3 / 3


,Title,Teaser,Link
0,Mars Missions,Since the first spacecraft was sent to Mars w...,https://space-facts.com/mars-missions/
1,Mars Curiosity Facts,With the Mars Curiosity landing just a few ho...,https://space-facts.com/mars-curiosity-facts/
2,Mars Panorama,A 360 degree panorama of the Martian landscap...,https://space-facts.com/mars-panorama/


---
### Mars Hemispheres
- Get hi-res images of each of Mar's hemispheres
- Grab the name and save the URL
- Store data as dict